In [1]:
import findspark
findspark.init('/home/dangkhoa/spark-2.3.1-bin-hadoop2.7')

## Session

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Logistic_Regression').getOrCreate()

## Load dataset

In [3]:
dataset = spark.read.format("libsvm").load("sample_libsvm_data.txt")

dataset.show(5)

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(692,[127,128,129...|
|  1.0|(692,[158,159,160...|
|  1.0|(692,[124,125,126...|
|  1.0|(692,[152,153,154...|
|  1.0|(692,[151,152,153...|
+-----+--------------------+
only showing top 5 rows



In [4]:
dataset \
    .select('label').distinct().show()

+-----+
|label|
+-----+
|  0.0|
|  1.0|
+-----+



## Split Train/Test set

In [5]:
train_data,test_data = dataset.randomSplit([0.7,0.3])

## Logistic Regression

In [6]:
from pyspark.ml.classification import LogisticRegression

lr = LogisticRegression()

In [7]:
# Train
model = lr.fit(train_data)

## Model summary

In [8]:
trainingSummary = model.summary

trainingSummary.predictions.show(5)

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(692,[95,96,97,12...|[21.0883514538431...|[0.99999999930586...|       0.0|
|  0.0|(692,[98,99,100,1...|[31.8028652571065...|[0.99999999999998...|       0.0|
|  0.0|(692,[100,101,102...|[18.4755845463433...|[0.99999999053423...|       0.0|
|  0.0|(692,[122,123,124...|[19.4935686176030...|[0.99999999657980...|       0.0|
|  0.0|(692,[122,123,148...|[20.5492968488134...|[0.99999999880998...|       0.0|
+-----+--------------------+--------------------+--------------------+----------+
only showing top 5 rows



## Evaluate Testset

In [9]:
test_results = model.evaluate(test_data)

test_results.predictions.show(5)

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(692,[121,122,123...|[22.0452440155993...|[0.99999999973339...|       0.0|
|  0.0|(692,[124,125,126...|[70.6109079061146...|[1.0,2.1581038235...|       0.0|
|  0.0|(692,[124,125,126...|[31.2494674208077...|[0.99999999999997...|       0.0|
|  0.0|(692,[124,125,126...|[26.3802623027742...|[0.99999999999650...|       0.0|
|  0.0|(692,[125,126,127...|[22.9831316002293...|[0.99999999989563...|       0.0|
+-----+--------------------+--------------------+--------------------+----------+
only showing top 5 rows



In [10]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

evaluator = BinaryClassificationEvaluator()

roc = evaluator.evaluate(test_results.predictions)

roc

1.0